In [0]:
%sql
USE CATALOG ecommerce;

In [0]:
%sql
SHOW TABLES IN gold;
SHOW TABLES IN silver;

database,tableName,isTemporary
silver,category_metrics,false
silver,events,false
silver,events_part,false
silver,funnel_overall,false
silver,ml_train_features,false
silver,weekend_conversion_summary,false


In [0]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.7/798.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 70.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install -U transformers torch mlflow
dbutils.library.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.3/597.3 kB 25.9 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.7.0
    Not uninstalling blinker at /usr/lib/python3/dist-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b1f76d90-3c4c-4d41-b25b-af87b6146ada
    Can't uninstall 'blinker'. No files were found to uninstall.
  Attempting uninstall: mlflow-skinny
    Found existing installation: mlflow-skinny 2.22.0
    Not uninstalling mlflow-skinny at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b1f76d90-3c4c-4d41-b25b-af87b6146ada
    Can't uninstall 'm

In [0]:
import mlflow
from transformers import pipeline

classifier = pipeline("sentiment-analysis")  # downloads a default sentiment model
reviews = ["This product is amazing!", "Terrible quality, waste of money"]
results = classifier(reviews)

results

/local_disk0/.ephemeral_nfs/envs/pythonEnv-b1f76d90-3c4c-4d41-b25b-af87b6146ada/lib/python3.12/site-packages/torch/_vmap_internals.py:9: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  from torch.utils._pytree import _broadcast_to_and_flatten, tree_flatten, tree_unflatten
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998860359191895},
 {'label': 'NEGATIVE', 'score': 0.9998160004615784}]

In [0]:
from collections import Counter

labels = [r["label"] for r in results]
counts = Counter(labels)

with mlflow.start_run(run_name="sentiment_model"):
    mlflow.log_param("pipeline", "sentiment-analysis")
    mlflow.log_param("library", "transformers")
    mlflow.log_metric("n_reviews", len(reviews))
    mlflow.log_metric("n_positive", counts.get("POSITIVE", 0))
    mlflow.log_metric("n_negative", counts.get("NEGATIVE", 0))

counts

Counter({'POSITIVE': 1, 'NEGATIVE': 1})